In [81]:
import pandas as pd

In [82]:
df = pd.read_csv("en openfoodfacts org products.csv")

In [83]:
df.drop(df[df['pnns_groups_1'] == "unknown"].index, inplace=True)

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4735 entries, 6971 to 13218
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   code                                     4735 non-null   int64  
 1   product_name                             4475 non-null   object 
 2   quantity                                 3582 non-null   object 
 3   packaging_en                             1798 non-null   object 
 4   brands_en                                4488 non-null   object 
 5   categories_en                            4653 non-null   object 
 6   origins_en                               888 non-null    object 
 7   labels_en                                2494 non-null   object 
 8   countries_en                             4735 non-null   object 
 9   nutriscore_grade                         4650 non-null   object 
 10  pnns_groups_1                            4650 non

In [85]:
df.dropna(subset=['pnns_groups_2'], inplace=True)

In [86]:
df.to_csv("cleaned_data.csv")

In [87]:
'''

df_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v1.csv', sep=';')

df_co2.dropna(subset=['category_name'], inplace=True)

df_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')

df_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)

df_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)

df_merged.sample(5)

'''

"\n\ndf_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v1.csv', sep=';')\n\ndf_co2.dropna(subset=['category_name'], inplace=True)\n\ndf_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')\n\ndf_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)\n\ndf_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)\n\ndf_merged.sample(5)\n\n"

In [88]:
# df_merged.to_csv("merged_data.csv")

In [89]:
df_main = pd.read_csv("merged_data.csv")
df_update = pd.read_csv("to_categories.csv")
 
df_main["code"] = df_main["code"].astype(str)
df_update["code"] = df_update["code"].astype(str)
 
df_main_updated = df_main.merge(df_update, on='code', how='left')
df_main_updated
 
# Update only where new categories exist
df_main_updated['pnns_groups_2'] = df_main_updated['new_category (pnns_groups_2)'].combine_first(df_main_updated['pnns_groups_2'])
 
# Drop the helper column
df_main_updated.drop(columns=['new_category (pnns_groups_2)'], inplace=True)
 
df_main_updated.to_csv("updated_products.csv", index=False)
 
df_main_updated.sample(3)

,Unnamed: 0,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
2674,2673,7350068291128,Protein pudding vanilla,500 g,NaN,"njie,propud","Dairies,Desserts,Dairy desserts,Entremets - mo...",NaN,"No added sugar,No lactose","Finland,Sweden",a,Milk and dairy products,Protein snacks and drinks,"Milk and dairy products,Dairy desserts",NaN,unknown,Vanilla creamy puddings,2.05
1177,1177,6408430023578,Lisäaineeton jogurtti mansikka,NaN,NaN,Valio,"Dairies,Fermented foods,Fermented milk product...",NaN,NaN,Finland,c,Milk and dairy products,Yogurt,"Milk and dairy products,Milk and yogurt",NaN,NaN,Strawberry yogurts,1.05
3475,3473,6415712905088,Riisipuuro,300g,Plastic,Kotimaista,"Plant-based foods and beverages,Plant-based fo...",Finland,No additives,Finland,c,Cereals and potatoes,Breakfast cereals,"Cereals and potatoes,Breakfast cereals",45.0,c,Porridge,2.65


In [90]:
df_main_updated['pnns_groups_2'].value_counts()

pnns_groups_2
Sweets                              343
One-dish meals                      292
Chocolate products                  238
Bread                               236
Biscuits and cakes                  234
Plant-based milk substitutes        221
Cheese                              201
Dressings and sauces                192
Milk and yogurt                     188
Yogurt                              188
Dairy desserts                      187
Breakfast cereals                   184
Protein snacks and drinks           158
Processed meat                      154
Cereals                             152
Ice cream                           132
Sweetened beverages                 121
Alcoholic beverages                  84
Fats                                 78
Poultry                              76
Vegetables                           71
Artificially sweetened beverages     70
Appetizers                           66
Pastries                             64
Canned foods              

In [91]:
df_main_updated.shape

(4652, 18)

In [92]:
df_main_updated.drop_duplicates(subset=['code'], inplace=True)
df_main_updated.shape

(4650, 18)

In [93]:


to_delete = pd.read_csv("to_delete.csv")

df_main_updated['code'] = df_main_updated['code'].astype(str)
to_delete['code'] = to_delete['code'].astype(str)
df_main_updated = df_main_updated[~df_main_updated['code'].isin(to_delete['code'])]

df_main_updated.shape


(4603, 18)

In [94]:
df_main_updated = df_main_updated.drop(columns=['categories_en', 'nutriscore_grade', 'pnns_groups_1'])
df_main_updated



,Unnamed: 0,code,product_name,quantity,packaging_en,brands_en,origins_en,labels_en,countries_en,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
0,0,16229916411,"Foco, 100% pure coconut water with mango",500 ml,NaN,"Foco,Thai-agri-foods-co-ltd",NaN,NaN,"Finland,United States",Unsweetened beverages,"Beverages,Unsweetened beverages",NaN,NaN,Coconut waters,0.55
1,1,640295013004,Paprika original Chips,NaN,NaN,Zweifel,NaN,NaN,"Finland,World",Appetizers,"Salty snacks,Appetizers",NaN,NaN,Crisps,2.10
2,2,644504010013,Toblerone,NaN,NaN,NaN,NaN,NaN,"Finland,France,United Kingdom",Chocolate products,"Sugary snacks,Chocolate products",NaN,unknown,Chocolates,2.95
3,3,20043032,Easter Egg Mix,NaN,NaN,Fresh-easy,NaN,NaN,"Finland,United States",Chocolate products,"Composite foods,One-dish meals",NaN,NaN,Soups,2.95
4,4,20246297,Butter Fudge toffee,150 g,NaN,hatherwood,NaN,NaN,"Finland,France",Sweets,"Sugary snacks,Sweets",NaN,unknown,Salted butter caramels,2.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4647,4645,6411402164902,Reilu,500 g,Plastic,Oululainen,NaN,NaN,Finland,Bread,"Cereals and potatoes,Bread",NaN,unknown,Breads,2.50
4648,4646,6411402578525,Imatran riisipiirakka,6 kpl,fi:kartonki-muovi,Fazer,NaN,NaN,Finland,Pastries,"Composite foods,Pizza pies and quiches",NaN,unknown,Quiches,2.95
4649,4647,6420256010860,NaN,120 g,"Plastic,Other-plastics",Cloetta,NaN,NaN,Finland,Sweets,"Sugary snacks,Sweets",NaN,unknown,Bonbons,2.95
4650,4648,6430053410015,Naan Tandoor - Leipä,250 g,fi:muovipussi,Kirkuk-leipomo,NaN,NaN,Finland,Bread,"Cereals and potatoes,Bread",NaN,unknown,Naans,2.50


In [95]:
to_name = pd.read_csv('to_name.csv')
to_name['code'] = to_name['code'].astype(str)
df_temp = df_main_updated.merge(to_name[['code', 'product_name']], on='code', how='left', suffixes=('', '_new'))
df_temp['product_name'] = df_temp['product_name_new'].combine_first(df_temp['product_name'])
df_temp.drop(columns=['product_name_new'], inplace=True)
df_main_updated = df_temp

In [96]:
new_column_order = ["code", "product_name", "brands_en", "quantity", "packaging_en", "origins_en", "labels_en",	"countries_en", "pnns_groups_2", "food_groups_en", "main_category_en", "environmental_score_score", "environmental_score_grade", "carbon-footprint_1kg"]
df_main_updated = df_main_updated[new_column_order]
df_main_updated.head(10)

,code,product_name,brands_en,quantity,packaging_en,origins_en,labels_en,countries_en,pnns_groups_2,food_groups_en,main_category_en,environmental_score_score,environmental_score_grade,carbon-footprint_1kg
0,16229916411,"Foco, 100% pure coconut water with mango","Foco,Thai-agri-foods-co-ltd",500 ml,NaN,NaN,NaN,"Finland,United States",Unsweetened beverages,"Beverages,Unsweetened beverages",Coconut waters,NaN,NaN,0.55
1,640295013004,Paprika original Chips,Zweifel,NaN,NaN,NaN,NaN,"Finland,World",Appetizers,"Salty snacks,Appetizers",Crisps,NaN,NaN,2.10
2,644504010013,Toblerone,NaN,NaN,NaN,NaN,NaN,"Finland,France,United Kingdom",Chocolate products,"Sugary snacks,Chocolate products",Chocolates,NaN,unknown,2.95
3,20043032,Easter Egg Mix,Fresh-easy,NaN,NaN,NaN,NaN,"Finland,United States",Chocolate products,"Composite foods,One-dish meals",Soups,NaN,NaN,2.95
4,20246297,Butter Fudge toffee,hatherwood,150 g,NaN,NaN,NaN,"Finland,France",Sweets,"Sugary snacks,Sweets",Salted butter caramels,NaN,unknown,2.95
5,20575557,Medium egg noodles,Combino,250 g,NaN,NaN,NaN,"Finland,France,United Kingdom",Cereals,"Cereals and potatoes,Cereals",Egg noodles,NaN,unknown,2.65
6,20683191,Cookie brownie,Mcennedy,8 x 30 g,NaN,NaN,NaN,"Finland,France,Spain",Biscuits and cakes,"Sugary snacks,Biscuits and cakes",Biscuits and cakes,NaN,unknown,2.50
7,20886561,Sweet liquorice lakritsijäätelö,Lidl,68 g,NaN,NaN,NaN,"Finland,Norway",Ice cream,"Milk and dairy products,Ice cream",fi:lakritsijäätelöpuikot,NaN,NaN,2.60
8,2244898002793,AH Vloerbrood meergranen,Albert Heijn,NaN,NaN,NaN,NaN,"Finland,Netherlands",Bread,"Cereals and potatoes,Bread",Breads,NaN,NaN,2.50
9,2244899002693,AH Vloerbrood Meergranen HL,Albert Heijn,NaN,NaN,NaN,NaN,"Finland,Netherlands",Bread,"Cereals and potatoes,Bread",Breads,NaN,NaN,2.50


In [97]:
df_main_updated['carbon-footprint_1kg'].isna().sum()

np.int64(0)

In [98]:
df_main_updated['pnns_groups_2'].value_counts()

pnns_groups_2
Sweets                              340
One-dish meals                      292
Chocolate products                  234
Bread                               232
Biscuits and cakes                  229
Plant-based milk substitutes        221
Cheese                              201
Dressings and sauces                192
Milk and yogurt                     188
Yogurt                              188
Dairy desserts                      185
Breakfast cereals                   183
Protein snacks and drinks           158
Processed meat                      154
Cereals                             151
Ice cream                           131
Sweetened beverages                 118
Fats                                 78
Alcoholic beverages                  77
Poultry                              76
Vegetables                           70
Artificially sweetened beverages     68
Appetizers                           64
Pastries                             63
Canned foods              

In [99]:
df_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v3.csv', sep=';')

df_co2.dropna(subset=['category_name'], inplace=True)

df_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')

df_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)

df_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)

In [100]:
df_main_updated = df_main_updated.rename(columns={"code": "Barcode", "product_name": "Product name", "brands_en": "Brand", "quantity": "Quantity","packaging_en": "Packaging material", "origins_en": "Origin", "labels_en": "Label", "countries_en": "Sale countries", "pnns_groups_2": "Product group", "food_groups_en": "Food group", "main_category_en": "Product main category", "environmental_score_score": "Environmental score", "environmental_scrore_grade": "Environmental grade", "carbon-footprint_1kg": "Carbon-footprint 1kg"})

df_main_updated.describe()

,Environmental score,Carbon-footprint 1kg
count,927.000000,4603.000000
mean,55.625674,4.633615
std,23.336007,8.091435
min,-15.000000,0.031500
25%,37.000000,2.050000
50%,56.000000,2.650000
75%,75.000000,2.950000
max,114.000000,43.330000


In [101]:
min_val = 0.031500
max_val = 10

df_main_updated["sustainable footprint score"] = (((max_val - df_main_updated["Carbon-footprint 1kg"]) / (max_val - min_val)) * 100).round(1)

df_main_updated.sample(2)

,Barcode,Product name,Brand,Quantity,Packaging material,Origin,Label,Sale countries,Product group,Food group,Product main category,Environmental score,environmental_score_grade,Carbon-footprint 1kg,sustainable footprint score
366,6410405119605,Kaakaotäytekeksi,K-menu,350 g,NaN,NaN,NaN,Finland,Biscuits and cakes,"Sugary snacks,Biscuits and cakes",Biscuits,40.0,d,2.50,75.2
1774,6430042025152,Big pro strawberry flavour high protein drink,Maitomaa,NaN,NaN,NaN,"FSC,FSC Mix,No lactose",Finland,Protein snacks and drinks,"Beverages,Sweetened beverages",Dairy drinks,NaN,NaN,0.55,94.8


In [102]:
df_main_updated.isna().sum()

Barcode                           0
Product name                     38
Brand                           148
Quantity                       1045
Packaging material             2821
Origin                         3721
Label                          2132
Sale countries                    0
Product group                     0
Food group                        0
Product main category             0
Environmental score            3676
environmental_score_grade      3231
Carbon-footprint 1kg              0
sustainable footprint score       0
dtype: int64

In [103]:
df_main_updated['Product group'].value_counts()

Product group
Sweets                              340
One-dish meals                      292
Chocolate products                  234
Bread                               232
Biscuits and cakes                  229
Plant-based milk substitutes        221
Cheese                              201
Dressings and sauces                192
Milk and yogurt                     188
Yogurt                              188
Dairy desserts                      185
Breakfast cereals                   183
Protein snacks and drinks           158
Processed meat                      154
Cereals                             151
Ice cream                           131
Sweetened beverages                 118
Fats                                 78
Alcoholic beverages                  77
Poultry                              76
Vegetables                           70
Artificially sweetened beverages     68
Appetizers                           64
Pastries                             63
Canned foods              

In [104]:
df_main_updated[df_main_updated['Product group'] == 'Teas and herbal teas and coffees']

6430078530439

6430078530439